In [1]:
%load_ext autoreload
%autoreload 2

More lemma and featurization exploration.  Attempts to read in the hash table and to compose types lookup.

- 220616: Currently holding on resolution of naming conventions for multiword symbols (camel case vs lisp case)
- 220616: For examples of lemma declarations, see data/pvs/prelude/sets_lemmas.json
- 220616: For example of a lambda constant decl, see data/pvs/prelude/notequal.json
- 220616: For proof obligation expressions in theories, see data/pvs/prelude/QuotientDefinition.json

In [42]:
from collections import Counter
from pprint import pprint
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from pprint import pprint
from sklearn.model_selection import train_test_split
from IPython.display import JSON
from tqdm import tqdm
import json

from coprover import PROJ_ROOT, RSC_ROOT, PVSLIB_ROOT

from featurizer import *

DATA_ROOT = Path(PROJ_ROOT, "data", "pvs", "pvslib")
json_files = list(DATA_ROOT.rglob("*.json"))

In [3]:
theory_files = []
proofsession_files = []

for json_fpath in tqdm(json_files):
    with open(json_fpath, 'r') as f:
        json_root = json.load(f)
        tag = json_root[TAG]
        if tag == MODULE_WITH_HASH or tag == MODULE_WITH_HASH2:
            theory_files.append(json_fpath)
        elif tag == PROOF_SESSION:
            proofsession_files.append(json_fpath)
        else:
            print("Unknown tag={}, file={}".format(tag, json_fpath))
    

100%|████████████████████████████████████████████████████████████████████████| 9839/9839 [03:02<00:00, 53.89it/s]


In [43]:
print(len(theory_files), len(proofsession_files))

625 9214


In [49]:
def read_typehash(root_obj):
    assert TYPE_HASH in root_obj or TYPE_HASH2 in root_obj
    if TYPE_HASH in root_obj:
        typehash_root = root_obj[TYPE_HASH]
    else:
        typehash_root = root_obj[TYPE_HASH2]
    if isinstance(typehash_root, dict):
        assert typehash_root[TAG][0] == "typelist"
    elif isinstance(typehash_root, list):
        return {} # Empty type listing 
    type_lookup = {}
    # Do a first pass to collect basic types, then do a second
    # pass to collect those operating off those types.
    # Only do two passes for now, and worry about more complicated
    # typing structures later.
    for type_ptr, type_entry in typehash_root[ENTRIES].items():
        if type_ptr == FUNCTIONTYPE:
            return FUNCTIONTYPE  # Unusual, is Lisp code
        if isinstance(type_entry, str):
            return FUNCTIONTYPE  # Unusual, is Lisp code
        type_tag = type_entry[TAG]
        if type_tag == TYPENAME:
            type_lookup[type_ptr] = type_entry[ID]
        else:
            type_lookup[type_ptr] = type_tag
    return type_lookup


import pdb
def read_theory(json_root):
    assert json_root[TAG] == MODULE_WITH_HASH or json_root[TAG] == MODULE_WITH_HASH2 
    type_lookup = read_typehash(json_root)
    theory_lookup = {}
    for module in json_root[MODULE]:
        if module[TAG] == THEORY:
            theory_id = module[ID]
            for decl in module[DECLARATIONS]:
                # lamdas
                if decl[TAG] == CONST_DECL and \
                  CONST_DEF in decl and \
                  decl[CONST_DEF] is not None and TAG in decl[CONST_DEF] and \
                  decl[CONST_DEF][TAG] == LAMBDA:
                    name = decl[NAME]
                    expression = process(decl[CONST_DEF][EXPRESSION], type_lookup=type_lookup)
                    theory_lookup[name] = bfs_flatten(expression)
                elif decl[TAG] == FORMULA_DECL or decl[TAG] == FORMULA_DECL2:
                    # Encompasses obligations and lemmas
                    name = decl[ID]
                    expression = process(decl[DEFINITION])
                    theory_lookup[name] = bfs_flatten(expression)
    return theory_lookup
    
def read_proof_session(json_root):
    assert PROOF in json_root and json_root[TAG] == PROOF_SESSION
    proof_seq = json_root[PROOF]
    sa_pairs = []
    if proof_seq is not None:
        type_lookup = read_typehash(json_root)
        for idx in range(0, len(proof_seq), 2):
            proof_step = proof_seq[idx]
            input_step = proof_seq[idx + 1]
            state_toks = bfs_flatten(process(proof_step, type_lookup=type_lookup))
            if isinstance(input_step, list):
                input_cmd = input_step[1]
            elif isinstance(input_step, dict):
                input_cmd = input_step[RULE]
            sa_pairs.append((state_toks, input_step))
    return sa_pairs


In [7]:
def process_json(json_root):
    if json_root[TAG] == PROOF_SESSION:
        return read_proof_session(json_root)
    elif json_root[TAG] == MODULE_WITH_HASH:
        return read_theory(json_root)

# Theory Processing

In [47]:
#json_fpath = theory_files[85]
json_fpath = Path(PROJ_ROOT, "data/pvs/pvslib/algebra/p_groups.json")
with open(json_fpath, 'r') as f:
    theory_root = json.load(f)
    theory_lookup = read_theory(theory_root)
    for name, expr in theory_lookup.items():
        print("Name:{}\nExpr:{}\n\n".format(name, " ".join([str(x) for x in expr])))

Name:imp_finite_groups_tcc1
Expr:operator group_def__groupp constantName group_def__groupp argument constantName sets__fullset


Name:alt_is_action_tcc1
Expr:bindings variableName g variableName h variableName k expression bindings variableName x1 expression operator operator right_left_cosets__left_cosets constantName right_left_cosets__left_cosets argument variableName g variableName k argument operator operator p_groups__alt constantName p_groups__alt argument variableName g variableName h variableName k argument variableName x1


Name:alt_is_action
Expr:bindings variableName g variableName h variableName k expression operator operator group_action__group_actionp constantName group_action__group_actionp argument variableName h operator right_left_cosets__left_cosets constantName right_left_cosets__left_cosets argument variableName g variableName k argument operator p_groups__alt constantName p_groups__alt argument variableName g variableName h variableName k


Name:fix_iff_subset
Ex

In [50]:
json_fpath = Path(PROJ_ROOT, "data/pvs/pvslib/algebra/ring_with_one_prime_ideal-proofs/prime_ideal_charac.json")
with open(json_fpath, 'r') as f:
    proof_root = json.load(f)
    sa_pairs = read_proof_session(proof_root)
    for (state, action) in sa_pairs:
        print("INPUT:{}\nSTATE:{}\n\n".format(action, state))

INPUT:{'tag': 'input', 'rule': 'skolem', 'arguments': ['1', ['P!1', 'R!1']]}
STATE:['current-goal', 'antecedents', 'null', 'consequents', 'new?', 'null', 'formula', 'bindings', 'functiontype', 'subtype', 'expression', 'operator', 'booleans__IMPLIES', 'argument', 'operator', 'booleans__AND', 'argument', 'operator', 'ring_def__commutative_ringp', 'argument', 'subtype', 'operator', 'booleans__AND', 'argument', 'operator', 'monad_def__monadp', 'argument', 'operator', 'sets__remove', 'argument', 'subtype', 'operator', 'ring_ideal_def__idealp', 'argument', 'functiontype', 'subtype', 'operator', 'booleans__IFF', 'argument', 'operator', 'ring_prime_ideal_def__prime_idealp', 'argument', 'functiontype', 'subtype', 'operator', 'integral_domain_with_one_def__integral_domain_w_onep', 'argument', 'operator', 'restrict__restrict', 'argument', 'operator', 'cosets_def__divide', 'argument', 'subtype', 'functiontype', 'asserted?', 'null', 'hidden', 'null']


INPUT:{'tag': 'input', 'rule': 'skolem!', 'arg

In [51]:
type(json_root)

dict

In [15]:
JSON(theory_root)

<IPython.core.display.JSON object>

# Proof Steps Processing

In [10]:
from coprover.feats.featurize_cmdpred import *

#json_fpath = Path(PVSLIB_ROOT, "algebra", "cartesian_product_finite-proofs", "cartesian_product_n_add_is_union.json")
json_fpath = Path(PROJ_ROOT, "data/pvs/prelude/extend_func_props-proofs/surjective_extend.json")
with open(json_fpath, 'r') as f:
    json_root = json.load(f)

step = json_root['proof'][0]
tagform = convert_dict2tags(step)
state_str = format_state(tagform)
print(state_str)

<ANT> <CONS> s-formula forall ['variable'] apply constant type-actual type-actual apply constant type-actual type-actual type-actual const-actual formal-constant ['variable'] <HID> 


In [11]:
steps = read_proof_session(json_root)

In [12]:
" ".join([str(x) for x in steps[0][0]])

'current-goal antecedents null consequents new? null formula bindings subtype expression operator functions__surjectivep argument operator extend__extend argument subtype asserted? null hidden null'

In [13]:
steps = read_proof_session(json_root)
" ".join([str(x) for x in steps[3][0]])

'current-goal antecedents new? null formula operator booleans__NOT argument bindings extend_func_props__R expression bindings subtype expression operator equalities__equal argument operator extend_func_props__f!1 argument subtype extend_func_props__R asserted? null consequents new? null formula bindings extend_func_props__R expression bindings extend_func_props__T expression operator equalities__equal argument operator operator extend__extend argument argument extend_func_props__T extend_func_props__R asserted? null hidden null'

In [40]:
[step for step in steps if step[1] == "lemma"]

[]

In [14]:
JSON(json_root[PROOF][0])

<IPython.core.display.JSON object>

In [15]:
for json_fpath in tqdm(proofsession_files):
    with open(json_fpath, 'r') as f:
        doc_root = json.load(f)
        read_proof_session(doc_root)

100%|███████████████████████████████████████████████████████████████████████| 1777/1777 [00:11<00:00, 157.36it/s]


In [ ]:
import collections

all_theories = collections.OrderedDict()
for json_fpath in tqdm(theory_files):
    with open(json_fpath, 'r') as f:
        doc_root = json.load(f)
        theory = read_theory(doc_root)
        assert len(set(theory.keys()).intersection(set(all_theories.keys()))) == 0
        all_theories.update(theory)

  0%|                                                                                    | 0/624 [00:00<?, ?it/s]

> /tmp/ipykernel_16415/1608546115.py(40)read_theory()
     38                 pdb.set_trace()
     39                 # lamdas
---> 40                 if decl[TAG] == CONST_DECL and \
     41                   CONST_DEF in decl and \
     42                   decl[CONST_DEF] is not None and TAG in decl[CONST_DEF] and \



ipdb>  decl


{'tag': 'formulaDecl', 'id': 'imp_derivatives_tcc1', 'label': 'obligation', 'definition': [{'tag': 'constant', 'constantName': 'deriv_domain_def__deriv_domainp', 'actuals': [{'tag': 'typeActual', 'type': 17347970}], 'type': 2614391051}], 'proof': {'tag': 'proofInfo', 'script': '("" (lemma "deriv_domain") (("" (propax) nil nil)) nil)', 'status': None}}


ipdb>  cont


...

{'tag': 'varDecl', 'id': 'f2', 'declaredType': [{'type': 3232366962}], 'type': 1496957645}


ipdb>  cont


> /tmp/ipykernel_16415/1608546115.py(38)read_theory()
     36             theory_id = module[ID]
     37             for decl in module[DECLARATIONS]:
---> 38                 pdb.set_trace()
     39                 # lamdas
     40                 if decl[TAG] == CONST_DECL and \



ipdb>  decl


{'tag': 'varDecl', 'id': 'fp', 'declaredType': [{'type': 3232366962}], 'type': 1496957645}
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
> /tmp/ipykernel_16415/1608546115.py(40)read_theory()
     38                 pdb.set_trace()
     39                 # lamdas
---> 40                 if decl[TAG] == CONST_DECL and \
     41                   CONST_DEF in decl and \
     42                   decl[CONST_DEF] is not None and TAG in decl[CONST_DEF] and \

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
> /tmp/ipykernel_16415/1608546115.py(38)read_theory()
     36             theory_id = module[ID]
     37             for decl in module[DECLARATIONS]:
---> 38                 pdb.set_trace()
     39                 # lamdas
     40                 if decl[TAG] == CONST_DECL and \

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
> /tmp/ipykernel_16415/1608546115.py(40)read_theory()
     38                 pdb.set_trace()
     39                 #

In [52]:
len(theory_files)

{}

In [34]:
set(theory.keys()).intersection(set(all_theories.keys()))

{'extensionality'}

In [35]:
theory['extensionality']

['bindings',
 'UnkType',
 'UnkType',
 'expression',
 'operator',
 'booleans__IMPLIES',
 'argument',
 'bindings',
 'UnkType',
 'expression',
 'operator',
 'equalities__equal',
 'argument',
 'operator',
 'UnkType',
 'argument',
 'UnkType',
 'operator',
 'UnkType',
 'argument',
 'UnkType',
 'operator',
 'equalities__equal',
 'argument',
 'UnkType',
 'UnkType']

In [36]:
all_theories['extensionality']

['bindings',
 'UnkType',
 'UnkType',
 'expression',
 'operator',
 'booleans__IMPLIES',
 'argument',
 'bindings',
 'UnkType',
 'expression',
 'operator',
 'booleans__IFF',
 'argument',
 'operator',
 'sets__member',
 'argument',
 'UnkType',
 'UnkType',
 'operator',
 'sets__member',
 'argument',
 'UnkType',
 'UnkType',
 'operator',
 'equalities__equal',
 'argument',
 'UnkType',
 'UnkType']

In [41]:
json_fpath

PosixPath('/home/fennel2/yeh/proj/CoProver/src/coprover/../../data/pvs/prelude/functions.json')